<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/vLLM_Inference_with_Marlin_for_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to convert a GPTQ model to the Marlin format.

It also benchmarks inference's throughput of Marlin GPTQ with vLLM, using Mistral 7B.

Marlin requires an Ampere GPU, or more recent. On Google Colab, only the A100 works.

First, install the following libraries:

In [ ]:
!pip install --upgrade transformers auto-gptq accelerate nvidia-ml-py3 optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB

Test whether Marlin is supported by your GPU. A function "print_gpu_utilization()" is also defined to measure the memory consumption in the following cells.

In [ ]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
  print("Your GPU supports Marlin!")
else:
  print("Your GPU doesn't support Marlin... You need an Ampere GPU or more recent (RTX 30xx/40xx, A100, H100, ...)")


from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
from pynvml import *
import time

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


GPTQ_MODEL = "kaitchup/Mistral-7B-v0.1-gptq-4bit"

Your GPU supports Marlin!


# Conversion to Marlin
We convert the GPTQ simply by loading it with AutoGPTQForCausalLM and passing "use_marlin=True".



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(GPTQ_MODEL, use_fast=True)
marlin_model = AutoGPTQForCausalLM.from_quantized(
      GPTQ_MODEL,
      use_marlin=True,
      device_map='auto')

save_dir = "Mistral-7B-v0.1-gptq-marlin-4bit"

marlin_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

WARNING - ignoring unknown parameter in config.json: dataset.
WARNING - ignoring unknown parameter in config.json: modules_in_block_to_quantize.
WARNING - ignoring unknown parameter in config.json: quant_method.
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
Overriding QuantLinear layers to use Marlin's QuantLinear...: 100%|██████████| 454/454 [00:37<00:00, 12.16it/s]
WARNING - you are using save_pretrained, which will re-direct to save_quantized.


('Mistral-7B-v0.1-gptq-marlin-4bit/tokenizer_config.json',
 'Mistral-7B-v0.1-gptq-marlin-4bit/special_tokens_map.json',
 'Mistral-7B-v0.1-gptq-marlin-4bit/tokenizer.model',
 'Mistral-7B-v0.1-gptq-marlin-4bit/added_tokens.json',
 'Mistral-7B-v0.1-gptq-marlin-4bit/tokenizer.json')

# Usage and benchmarking with vLLM

In [ ]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Marlin's throughput is benchmarked forbatch sizes: 1, 2, 4, 8, 16, 32, 64, and 128.

In [ ]:
import time
from vllm import LLM, SamplingParams

batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128]
p = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n\n Tell me about gravity."

sampling_params = SamplingParams(max_tokens=1000)

loading_start = time.time()
llm = LLM(model="kaitchup/Mistral-7B-v0.1-gptq-marlin-4bit")
print("--- Loading time: %s seconds ---" % (time.time() - loading_start))


for b in batch_sizes:
  prompts = []
  for i in range(b):
    prompts.append(p)

  generation_time = time.time()
  outputs = llm.generate(prompts, sampling_params)
  duration = time.time() - generation_time
  total_tokens = 0
  for output in outputs:
    total_tokens += len(output.prompt_token_ids) + len(output.outputs[0].token_ids)
  print('\nBatch size: '+str(b))
  print("--- Speed: %s tokens/second ---" % (round(total_tokens/duration,2)))



config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

INFO 03-23 01:51:43 llm_engine.py:87] Initializing an LLM engine with config: model='kaitchup/Mistral-7B-v0.1-gptq-marlin-4bit', tokenizer='kaitchup/Mistral-7B-v0.1-gptq-marlin-4bit', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=marlin, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...


Batch size: 32
--- Speed: 1623.01 tokens/second ---


Processed prompts: 100%|██████████| 64/64 [00:09<00:00,  6.93it/s]



Batch size: 64
--- Speed: 2431.62 tokens/second ---


Processed prompts: 100%|██████████| 128/128 [00:15<00:00,  8.41it/s]


Batch size: 128
--- Speed: 3431.21 tokens/second ---


Benchmarking the original GPTQ model for reference, using the same code:

In [ ]:
import time
from vllm import LLM, SamplingParams

batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128]
p = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n\n Tell me about gravity."

sampling_params = SamplingParams(max_tokens=1000)

loading_start = time.time()
llm = LLM(model="kaitchup/Mistral-7B-v0.1-gptq-4bit")
print("--- Loading time: %s seconds ---" % (time.time() - loading_start))


for b in batch_sizes:
  prompts = []
  for i in range(b):
    prompts.append(p)

  generation_time = time.time()
  outputs = llm.generate(prompts, sampling_params)
  duration = time.time() - generation_time
  total_tokens = 0
  for output in outputs:
    total_tokens += len(output.prompt_token_ids) + len(output.outputs[0].token_ids)
  print('\nBatch size: '+str(b))
  print("--- Speed: %s tokens/second ---" % (round(total_tokens/duration,2)))



WARNING 03-23 01:59:30 config.py:193] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-23 01:59:30 llm_engine.py:87] Initializing an LLM engine with config: model='kaitchup/Mistral-7B-v0.1-gptq-4bit', tokenizer='kaitchup/Mistral-7B-v0.1-gptq-4bit', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 03-23 01:59:34 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 03-23 01:59:38 llm_engine.py:357] # GPU blocks: 14035, # CPU blocks: 2048
INFO 03-23 01:59:40 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.69it/s]



Batch size: 1
--- Speed: 464.51 tokens/second ---


Processed prompts: 100%|██████████| 2/2 [00:06<00:00,  3.17s/it]



Batch size: 2
--- Speed: 208.45 tokens/second ---


...


Batch size: 32
--- Speed: 924.51 tokens/second ---


Processed prompts: 100%|██████████| 64/64 [00:18<00:00,  3.44it/s]



Batch size: 64
--- Speed: 1266.42 tokens/second ---


Processed prompts: 100%|██████████| 128/128 [00:30<00:00,  4.25it/s]


Batch size: 128
--- Speed: 1676.06 tokens/second ---
